In [1]:
from IPython.core.display import display, HTML
#makes the display bars longer and spread out 100% across the screen width
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import nathaniel.model_validation.vivarium_output_loader as vol
import nathaniel.model_validation.ciff_output_processing as cop

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__
ndbs
Fri Jul 30 14:31:21 PDT 2021


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!ls /ihme/homes/ndbs/vivarium_results/ciff_malnutrition/verification_and_validation/

v2.1_gbd_2020_wasting  v2.3_wasting_birth_prevalence


In [5]:
os.path.abspath('~/vivarium_results')

'/ihme/homes/ndbs/vivarium_research_ciff_sam/nathaniel/scratch__gitignore__/~/vivarium_results'

# Define directories

In [6]:
# sim results path:
# /ihme/costeffectiveness/results/vivarium_ciff_sam/v2.3_wasting_birth_prevalence/ciff_sam/2021_07_26_17_14_31/count_data

project_results_dir = '/ihme/costeffectiveness/results/vivarium_ciff_sam'
model_name = 'v2.3_wasting_birth_prevalence'
model_timestamp = '2021_07_26_17_14_31'

model_count_data_dir = f'{project_results_dir}/{model_name}/ciff_sam/{model_timestamp}/count_data/'

project_vv_directory_name = 'ciff_malnutrition/verification_and_validation'
username = 'ndbs'

output_dir = f'/ihme/homes/{username}/vivarium_results/{project_vv_directory_name}/{model_name}'

share_output_dir = f'/share/scratch/users/ndbs/vivarium_results/{project_vv_directory_name}/{model_name}'
j_output_dir = f'/home/j/Project/simulation_science/{project_vv_directory_name}/{model_name}'

# Load transformed count data

In [7]:
data = vol.load_transformed_count_data(model_count_data_dir)
data.keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count'])

# Write a small class to wrap the count data dictionary and provide more convenient access via attributes

In [8]:
class VivariumCountData:
    def __init__(self, table_dict):
        for table_name, table in table_dict.items():
            setattr(self, table_name, table)
    
    def to_dict(self):
        return dict(self.__dict__)
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def table_names(self):
        return list(self.__dict__.keys())

d = VivariumCountData(data)
print(d.__dict__ is d.to_dict())
d.table_names()

False


['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [9]:
d.to_dict().keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count'])

# Alternate version of class to store Vivarium count data

In [61]:
class CountDataDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for key, value in self.items():
            setattr(self, key, value)
            
    def __setattr__(self, key, value):  
        super().__setattr__(key, value)
        if key not in self: # if statement needed to avoid infinite recursion...
            self[key] = value
    
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        setattr(self, key, value)
    
    def table_names(self):
        return list(self.keys())

c = CountDataDict(data)
c.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [62]:
c.__dict__.keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count'])

In [63]:
c.keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count'])

In [64]:
dir(c)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'clear',
 'copy',
 'deaths',
 'disease_state_person_time',
 'disease_transition_count',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'population',
 'setdefault',
 'table_names',
 'update',
 'values',
 'wasting_state_person_time',
 'wasting_transition_count',
 'ylds',
 'ylls']

In [65]:
c.ylls

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,ylls,29,baseline,88.871895,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,ylls,946,baseline,0.000000,severe_acute_malnutrition,2_to_4


In [66]:
c['empty_table'] = pd.DataFrame()
c.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table']

In [68]:
# Hmm, this doesn't work -- 'yak' doesn't get recorded in self.keys()
# Ok, now it works after I implemented __setattr__
c.yak = 2

In [69]:
c['yak']

2

In [70]:
# However, the bad key 1 made it into our keys() before the failure. Not good.
c.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 1,
 'yak']

In [74]:
c[1]

""


In [75]:
type(c[1])

pandas.core.frame.DataFrame

In [71]:
dir(c)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'clear',
 'copy',
 'deaths',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'population',
 'setdefault',
 'table_names',
 'update',
 'values',
 'wasting_state_person_time',
 'wasting_transition_count',
 'yak',
 'ylds',
 'ylls']

In [26]:
c.__dict__.keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count', 'empty_table', 'yak'])

In [76]:
import collections

class CountData2(collections.abc.MutableMapping):
    def __init__(self, table_dict):
        self.__dict__ = dict(table_dict)
#         for table_name, table in table_dict.items():
#             self[table_name] = table
    
#     def __setattr(self, key, value):
#         if not isinstance(value, pd.DataFrame):
#             raise TypeError(f"Only DataFrames are allowed in CountData. You provided {type(value)}")
#         super().__setattr__(key, value)
    
    def __setitem__(self, key, value):
        setattr(self, key, value)
        
    def __delitem__(self, key):
        del self.key
    
    def __getitem__(self, key):
        return self.__dict__[key]
    
    def __iter__(self):
        return iter(self.__dict__)
    
    def __len__(self):
        return len(self.__dict__)
    
    def to_dict(self):
        # return dict(self) # this also works since we implement Mapping
        return dict(self.__dict__)
    
    def table_names(self):
        return list(self.keys())

cd2 = CountData2(data)
print(cd2.__dict__ is cd2.to_dict())
cd2.table_names()

False


['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [82]:
cd2.deaths

,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,female,2022,diarrheal_diseases,death,29,baseline,1.0,susceptible_to_child_wasting,early_neonatal
1,female,2022,diarrheal_diseases,death,29,baseline,0.0,mild_child_wasting,early_neonatal
2,female,2022,diarrheal_diseases,death,29,baseline,0.0,moderate_acute_malnutrition,early_neonatal
3,female,2022,diarrheal_diseases,death,29,baseline,0.0,severe_acute_malnutrition,early_neonatal
...,...,...,...,...,...,...,...,...,...
17276,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,susceptible_to_child_wasting,2_to_4
17277,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,mild_child_wasting,2_to_4
17278,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,moderate_acute_malnutrition,2_to_4
17279,male,2026,severe_acute_malnutrition,death,946,baseline,0.0,severe_acute_malnutrition,2_to_4


In [77]:
cd2.empty_table = pd.DataFrame()

In [78]:
cd2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table']

In [83]:
cd2.empty_table

""


In [84]:
pd.DataFrame()

""


In [85]:
type(_)

pandas.core.frame.DataFrame

In [86]:
cd2.to_dict().keys()

dict_keys(['wasting_transition_count', 'wasting_state_person_time', 'deaths', 'population', 'ylls', 'ylds', 'disease_state_person_time', 'disease_transition_count', 'empty_table'])

In [87]:
cd2['goat'] = pd.DataFrame([1,2,3])

In [88]:
cd2.goat

,0
0,1
1,2
2,3


In [89]:
cd2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'goat']

In [90]:
cd2.__dict__.update({'a': 1, 'b': 2})
cd2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'goat',
 'a',
 'b']

In [91]:
cd2.a

1

In [92]:
del cd2.b
cd2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'goat',
 'a']

In [93]:
'a' in cd2

True

In [94]:
dict(cd2) is cd2.__dict__

False

In [42]:
list(cd2.keys())

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'goat',
 'a']

In [73]:
# Now we don't have the bad key 1 in keys()
cd2.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'empty_table',
 'goat',
 'a']

In [43]:
isinstance(pd.Series, np.ndarray)

False

In [45]:
isinstance(pd.Index, np.ndarray)

False